> **AutoGen > Use Cases > Enhanced Inference**
https://microsoft.github.io/autogen/0.2/docs/Use-Cases/enhanced_inference


# Enhanced Inference

    `autogen.OpenAIWrapper`는 `opnai>=1` 버전에서 **고급 LLM 추론 기능**을 제공한다. `autogen.Completion`은 `openai.Completion` 및 `openai.ChatCompletion`의 대체제로, `openai<1`에서 고급 LLM 추론을 위한 드롭인 방식의 대안이다.
    AUtoGen을 이용해 추론을 수행하면 다음과 같은 다양한 이점을 누릴 수 있다.
    **성능 조정**, **API 통합**, **캐싱**, **오류 처리**, **다중 설정 추론**, **결과 필터링**, **템플릿 처리** 등.

## Tune Inference Parameters(for openai <1)

    openai<1 용 추론 파라미터 조정의 예시 목록은 다음 페이지에서 확인 가능하다
> **Tune Inference Parameters Examples**
https://microsoft.github.io/autogen/0.2/docs/Examples#inference-hyperparameters-tuning

### Choices to optimize

    최적화 가능한 항목들을 살펴보면, Foundation Model(파운데이션 모델)을 사용한 텍스트 생성의 비용은 일반적으로 **입력과 출력 토큰 수의 총합**으로 측정된다. 파운데이션 모델을 사용하는 애플리케이션 개발자의 관점에서 보면, **정해진 추론 예산(예: 코딩 문제 하나를 푸는데 평균적으로 소모되는 비용)** 내에서 **생성된 텍스트의 유용성을 극대화**하는 것이 목표이다.
    이 목표는 **추론 하이퍼파라미터**를 최적화함으로써 달성할 수 있으며, 이는 생성 결과의 유용성과 비용에 큰 영향을 준다.
    조정 가능한 주요 하이퍼파라미터들을 살펴보면,

    (1) `model` : 사용할 모델 ID를 지정하는 필수 항목
    (2) `prompt/messages` : 모델에 입력되는 프롬프트 또는 메시지. 생성 작업의 맥락을 제공한다.
    (3) `max_tokens` : **생성될 최대 토큰 수**
    (4) `temperature`: 생성 결과의 **무작위성을 조절**하는 값(0~1). 값이 높을수록 다양하고 창읮거인 결과이고 낮을 수록 예측 가능한 결과가 나온다.
    (5) `top_p` : 각 토큰 생성 시 고려할 **누적 확률 질량의 범위를 설정**하는 값(0~1). 값이 낮으면 더 확실한 토큰만 고려하고 높으면 다양성을 높일 수 있다.
    (6) `n` : **프롬프트당 생성할 응답 수**. 응답 수가 많을 수록 다양성 증가, 하지만 비용도 함께 증가한다.
    (7) `stop` : 출력 중 **해당 문자열이 생성되면 중단**되도록 설정한다.
    (8) `presence_penalty`, `frequency_penalty` : **특정 단어나 구의 존재 및 반복에 대한 생성 영향력**을 조절한다.
    (9) `best_of` : 여러 응답 중 **가장 확률이 높은 결과**를 선택하도록 서버 측에서 응답을 여러 개 설정한다. 


    하이퍼파라미터 간의 상호작용 및 주의점으로는 `temperature` 와 `top_p`는 둘 다 생성의 무작위성을 조절하기 떄문에 같이 조정하면 **충돌**할 수 있다. 또한 `n`과 `best_of`는 일반적으로 **같이 조정하지 않는 것이 좋다.** `n`과 `max_tokens`는 **전체 생성 토큰 수**에 영향을 주므로, 결과적으로 비용에 영향을 준다.
    이러한 상호작용과 trade-off(트레이드 오프) 때문에 **수작업으로 최적의 하이퍼파라미터**를 찾는 것은 매우 어렵다.

    이러한 선택들이 실제로 중요한지에 대해서 아래 블로그 글에서의 gpt-3.5-turbo, gpt-4에 대한 튜닝 결과 예시를 확인 가능하다.
> **blogpost(exapmle tuning results)**
    https://microsoft.github.io/autogen/0.2/blog/2023/04/21/LLM-tuning-math/

    AutoGen으로 추론을 튜닝할 때는 다음 요소들이 필요하다.
    (1) `Validiation data` **검증 데이터**
    (2) `Evaluation function` **평가 함수**
    (3) `Metric` **최적화할 지표**
    (4) `Search space` **탐색 공간**
    (5) `Budgets` **예산 (추론 및 최적화)**



### Validation data (검증 데이터)
다양한 인스턴스를 수집하여  `dict` 형식의 반복 가능한 객체로 저장한다. 
이러한 인스턴스는 dict의 반복 가능한 객체에 저장할 수 있다. 예를 들어, 각 인스턴스 `dict`는 "problem"을 키로, 수학 문제의 설명 str을 값으로, "solution"을 키로, 해답 str을 값으로 포함할 수 있다.

```python
{"problem": "수학 문제 설명", "solution": "해답"}
```



### Evaluation function (평가 함수)
평가 함수는 응답 리스트와 검증 데이터의 키워드 인자를 받아 `성능 지표(metrics)`가 담긴 `dict`을 반환해야 한다.

`autogen.code_utils` 와 `autogen.math_utils`는 코드 생성 및 수학 문제 풀이를 위한 예시 평가 함수들을 제공한다.

### Metric to optimize

최적화할 지표는 일반적으로 튜닝에 사용되는 **모든 데이터 인스턴스에 대한 집계된 메트릭**이다. 예를 들어, 사용자는 `"suceess"`를 지표로 하고 최적화 모드를 `"max"`로 설정할 수 있다. 기본적으로 평균(average)를 집계 함수로 사용한다. 사용자가 필요하다면, **커스터마이즈된 집계 함수**를 직접 제공할 수 있다.

### Search space (탐색 공간)
사용자는 각 하이퍼파라미터에 대해 **선택적**으로 탐색 범위를 지정할 수 있다.

(1) `model` 
- 고정 문자열(str)로 지정하거나, `flaml.tune.choice`를 사용해 여러 선택지를 제공할 수 있다.
(2) `prompt / messages` 
- *prompt*는 문자열 또는 문자열 리스트로서, 프롬프트 템플릿을 의미한다.
- *messages*는 딕셔너리의 리스트 또는 딕셔너리 리스트들의 리스트 형태로, 메시지 템플릿을 나타낸다.
- 각 프롬프트/메시지 템플릿은 각각의 데이터 인스턴스에 따라 포맷팅된다.
예를 들어 프롬프트 템플릿은 다음과 같을 수 있다.

```script
"{problem} Solve the problem carefully. Simplify your answer as much as possible. Put the final answer in \boxed{{}}."
```
이 경우 {problem}은 각 데이터 인스턴스의 "problem" 필드로 대체된다.

(3) `max_tokens`, `n`, `best_of` 
- 이들은 고정된 값으로 지정하거나, `flaml.tune.randint`, `flaml.tune.qrandint`, `laml.tune.lograndint`, `flaml.qlograndint` 등을 사용하여 탐색 범위를 지정할 수 있다.
- 기본적으로 `max_tokens`는 `[50, 1000)` 범위에서 탐색되며 `n`은 `[1, 100)` 범위에서 탐색된다.
- `best_of`는 기본적으로 1로 고정된다.

(4) `stop` 
0 문자열(str), 문자열 리스트, 문자열 리스트들의 리스트, 또는 `None`일 수 있다. 기본값은 	`None`이다.

(5) `temperature` 또는 `top_p`
- 이 둘중 하나는 `flaml.tune.uniform`, `flaml.tune.loguniform` 등으로 지정하거나 고정된 값으로 설정할 수 있다.
- **둘은 동시에 지정하지 않는다**
- 기본적으로 `[0,1]` 구간에서 `temperatrue` 또는 `top_p` 중 하나를 임의로 선택하여 구성한다.

(6) `presence_penalty`, `frequency_penalty` 
- 고정값으로 지정하거나 `flaml.tune.uniform`을 통해 설정할 수 있다.
- 기본적으로는 튜닝되지 않는다.


### Budgets
**Inference budget**(`추론 예산`) 은 데이터 인스턴스당 평균 추론 비용을 의미한다.
**Optimization budget**(`최적화 예산`)은 튜닝 과정 전체에서 사용할 수 있는 총 예산을 의미한다. 두 예산 모두 달러 기준, **1000 토큰당 가격을 따른다.**

### Perform tuning

이제 `autogen.Completion.tune`을 사용해 튜닝을 수행할 수 있다.
예를 들어 `num_samples`는 샘플링할 설정의 수를 의미한다.

## API unification

`autogen.OpenAIWrapper.create()`는 **chat**(채팅모델)과 **non-chat**(비채팅 모델) 모두에 대해 생성(completion)을 만들 수 있으며 **OpenAI API**와 **Azure OpenAI API** 양쪽 모두에서 사용할 수 있다.


In [4]:
from autogen import OpenAIWrapper

client = OpenAIWrapper()
response = client.create(messages=[
    {
        "role" : "user",
        "content" : "2+2=",
    }
],
                         model='gpt-3.5-turbo')

print(client.extract_text_or_completion_object(response))

['4']


In [5]:
print(response.cost)

8.5e-06


## Usage Summary
    `autogen`의 `OpenAIWrapper`는 API 호출 시 **토큰 수 및 비용**을 추적한다.
    `create()` 메서드를 사용하여 요청을 시작하고, `print_usage_summary()`를 호출하여 전체 사용 내역(캐시된 요청과 실제 요청 모두 포함)의 **비용 및 토큰 사용량**을 확인할 수 있다.

    - `model=["actual", "total"]` (기본값) : **모든 생성 요청 및 캐시되지 않은 요청에 대한 사용 요약을 출력**한다.
    - `model='actual'` : **캐시되지 않은 요청**만 출력한다.
    - `model='total` : **전체 요청 (캐시 포함)**에 대한 사용 정보를 출력한다.

In [6]:
from autogen import OpenAIWrapper

client= OpenAIWrapper()
client.create(
    messages=[
        {
            "role" : "user",
            "content" : "Python learnign tips."
        }
    ],
    model = "gpt-3.5-turbo",
)

client.print_usage_summary()
client.clear_usage_summary()

----------------------------------------------------------------------------------------------------
Usage summary excluding cached usage: 
Total cost: 0.00044
* Model 'gpt-3.5-turbo-0125': cost: 0.00044, prompt_tokens: 12, completion_tokens: 287, total_tokens: 299

All completions are non-cached: the total cost with cached completions is the same as actual cost.
----------------------------------------------------------------------------------------------------


### Error handling

### Runtime error

서로 다른 `모델/엔드포인트의 설정 목록`을 전달하여 **속도 제한(rate litmit)**이나 기타 런타임 오류를 완화할 수 있다. 
예를 들어

In [8]:
from autogen import OpenAIWrapper
from config import settings

In [9]:
open_ai_api_key = settings.openai_api_key.get_secret_value()
anthropic_api_key = settings.anthropic_api_key.get_secret_value()

In [12]:
client = OpenAIWrapper(
    config_list=[
        {
            "model" : "gpt-4o-mini",
            "api_key": open_ai_api_key,
            
        },
        {
            "model" : "gpt-3.5-turbo",
            "api_key" : open_ai_api_key,
            
        },
        {
            "model" : "claude-3-5-haiku-20241022",
            "api_key" : anthropic_api_key,
            
        },
    ]
)

### Logic error

또 다른 유형의 오류는 **응답이 특정 요구 조건을 만족하지 못하는 경우**이다. 예를 들어 응답이 **유효한 JSON 문자열** 이여야 할 때, 그렇지 않은 응답은 필터링하고 싶을 수 있다.
이를 위해 `a list of configurations`(설정 목록) 과 `a filter function`(필터 함수)를 제공할 수 있다.


In [22]:
import json
from autogen import OpenAIWrapper

config_list = [
    {
        "model": "text-ada-001",
    },
    {
        "model" : "gpt-3.5-turbo-instruct"
    },
]

In [23]:
def valid_json_filter(response, **_):
    for text in OpenAIWrapper.extract_text_or_completion_object(response):
        try:
            json.loads(text)
            return True
        except ValueError:
            pass
    return False

client = OpenAIWrapper(
    config_list=config_list,
)
response = client.create(
    prompt="How to construct a json request to Bing API to search for 'latest AI news'? Return the JSON request.",
    filter_func=valid_json_filter,
)

In [24]:
response

Completion(id='cmpl-BPglxEpdWNN4q1X8GDrgY0rSNGdNr', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text="\nThe JSON request to Bing API for searching 'latest AI news' would look")], created=1745462317, model='gpt-3.5-turbo-instruct:20230824-v2', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=22, total_tokens=38, completion_tokens_details=None, prompt_tokens_details=None), cost=0, message_retrieval_function=<bound method OpenAIClient.message_retrieval of <autogen.oai.client.OpenAIClient object at 0x11f7c2990>>, config_id=1, pass_filter=False)

## Templating

제공된 prompt나 message가 템플릿인 경우, 주어진 context(문맥)을 바탕으로 자동으로 완성된다.

In [52]:
from autogen import OpenAIWrapper
from config import settings

api_key = settings.openai_api_key.get_secret_value()

llm_config = [
    {
        "model" : "gpt-4o-mini",
        "api_key" : api_key,
    }
]

config =  {
        "model" : "gpt-4o-mini",
    }

client = OpenAIWrapper(
    config_list=config_list
)

In [53]:
response = client.create(
    context={"problem" : "How many positive integers, not exceeding 100, are multiples of 2 or 3 but not 4?"},
    prompt= "{problem} Solve the problem carefully",
    allow_format_str_template=True,
    **config,
)

In [54]:
response

Completion(id='cmpl-BPgvwKWTP6VxKeYP4QPYyfQiCjJNx', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text='.\n\nTo solve this problem, we need to first identify the multiples of ')], created=1745462936, model='gpt-3.5-turbo-instruct:20230824-v2', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=27, total_tokens=43, completion_tokens_details=None, prompt_tokens_details=None), cost=0, message_retrieval_function=<bound method OpenAIClient.message_retrieval of <autogen.oai.client.OpenAIClient object at 0x1287a34d0>>, config_id=1, pass_filter=True)